In [2]:
import treecorr
import pyccl as ccl
import numpy as np

# Columns ordering:
##'ra','dec','z','g1,'g2','weight','delta','e_obs_1','e_obs_2', 'e_obs_no_noise_1','e_obs_no_noise_2','e_IA_TATT_1', 'e_IA_TATT_2'

#pure shear signal, noisefree:
config_G = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 3,
          'g2_col' : 4,
          'w_col' : 5,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}

# observed ellipticities, with shear, noise and IA
config_IA = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 8,
          'g2_col' : 9,
          'w_col' : 6,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}

# observed ellipticities with shear, IA, noisefree
config_IA_noisefree = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 10,
          'g2_col' : 11,
          'w_col' : 6,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}


# pure IA
config_pureIA = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 12,
          'g2_col' : 13,
          'w_col' : 6,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}



We load all of the data into a single array for simplicity, n=6 is used for consistency so that we can start counting from 1 to 5 instead of from 0 to 4

Here we calculate the auto correlation functions for all bins

In [3]:
for i in np.arange(5)+1:
    #filename = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i)
    filename = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.dat'.format(i)
    cat = treecorr.Catalog(filename, config_G, flip_g2=True)
    #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    #gg.process(cat)
    gg.process(cat, metric = 'Arc')
    gg.write('correlations/cross_correlation_tomo{}_{}_linear_bias'.format(i,i))
    #gg.write('correlations/cross_correlation_tomo{}_{}'.format(i,i))

    print('Done tomo', i)

Done tomo 1
Done tomo 2
Done tomo 3
Done tomo 4
Done tomo 5


Here we calculate the cross correlation functions for all bins

In [3]:
gg.xip

array([1.00045588, 1.00037944, 1.0002824 , 1.00022363, 1.00016733,
       1.00012737, 1.00009047, 1.00005798, 1.00002227, 0.99996944,
       0.99989671, 0.9997696 , 0.99955227, 0.99918221, 0.9985295 ,
       0.99738978, 0.99538037, 0.991948  , 0.98618924, 0.97657397])

In [5]:
for i in np.arange(5)+1:
    for j in range(1, i):
        #filename1 = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i)
        #filename2 = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(j)

        filename1 = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.dat'.format(i)
        filename2 = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.dat'.format(j)
        cat_1 = treecorr.Catalog(filename1, config_G, flip_g2=True)
        cat_2 = treecorr.Catalog(filename2, config_G, flip_g2=True)
        #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2,metric = 'Arc')
        #gg.process(cat_1, cat_2, metric = 'Arc')
        #gg.write('correlations/cross_correlation_tomo{}_{}'.format(j,i))
        gg.write('correlations/cross_correlation_tomo{}_{}_linear_bias'.format(j,i))

        print('Done', i,j)

Done 2 1
Done 3 1
Done 3 2
Done 4 3
Done 5 1
Done 5 2
Done 5 3
Done 5 4


Here we show how to read the correlation functions saved to a file by TreeCorr

In [2]:
gg = treecorr.GGCorrelation(nbins = 20,
                       min_sep = 0.03333333, #2 arcmin
                       max_sep = 1.66666667)
gg.read('correlations/cross_correlation_tomo3_3')

In [2]:
from astropy.table import Table

In [8]:
#Compute xi+- for observed ellipticities with cosmic shear, noise and IA:

# Run the auto-correlation functions:
for i in np.arange(5)+1:
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
    filename = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)


    cat = treecorr.Catalog(filename, config_IA, flip_g2=True,first_row=2)
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    #gg.write('correlations/infused_cross_correlation_tomo{}_{}'.format(i,i))
    gg.write('correlations/infused_cross_correlation_tomo{}_{}_linear_bias'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_tomo_delta-NLA{}_{}'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_tomo_TATT{}_{}'.format(i,i))

    print("Done tomo", i)
    

Done tomo 1
Done tomo 2
Done tomo 3
Done tomo 4
Done tomo 5


In [9]:
# And the cross-redshift bins
for i in np.arange(5)+1:
    for j in range(1, i):
        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(j)

        filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)
        filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(j)


        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(j)

        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(j)

        cat_1 = treecorr.Catalog(filename1, config_IA, flip_g2=True,first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_IA, flip_g2=True,first_row=2)
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2,metric = 'Arc')
        #gg.write('correlations/infused_cross_correlation_tomo{}_{}'.format(j,i))
        gg.write('correlations/infused_cross_correlation_tomo{}_{}_linear_bias'.format(j,i))
        #gg.write('correlations/infused_cross_correlation_tomo_delta-NLA{}_{}'.format(j,i))
        #gg.write('correlations/infused_cross_correlation_tomo_TATT{}_{}'.format(j,i))

        print("Done tomo", i,j)

Done tomo 2 1
Done tomo 3 1
Done tomo 3 2
Done tomo 4 1
Done tomo 4 2
Done tomo 4 3
Done tomo 5 1
Done tomo 5 2
Done tomo 5 3
Done tomo 5 4


In [10]:
# Run treecorr on the mocks with IA infused, no noise:

# Run the auto-correlation functions:
#for i in range(1,2):
for i in range(1,6):
#for i in np.arange(5)+1:

    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
    filename = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)

    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_orig.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_orig_smooth.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_orig_smooth_2.5.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_0.0_0.0_1.0_7.5_{}.dat'.format(i)

    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_m5.0.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)

    cat = treecorr.Catalog(filename, config_IA_noisefree, flip_g2=True,first_row=2)
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo{}_{}'.format(i,i))
    gg.write('correlations/infused_cross_correlation_no_noise_tomo{}_{}_linear_bias'.format(i,i))

    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_delta-NLA{}_{}'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_delta-NLA{}_{}_Vm5.0'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_delta-NLA{}_{}_orig'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_delta-NLA{}_{}_orig_smooth'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_delta-NLA{}_{}_orig_smooth_2.5'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_TATT_0.0_0.0_1.0_7.5_{}_{}'.format(i,i))
    #gg.write('correlations/infused_cross_correlation_no_noise_tomo_TATT{}_{}'.format(i,i))

    print("Done tomo", i)



Done tomo 1
Done tomo 2
Done tomo 3
Done tomo 4
Done tomo 5


In [11]:
# And the cross-redshift bins
for i in np.arange(5)+1:
    for j in range(1, i):
        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(j)

        filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)
        filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(j)


        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(j)

        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(j)

        cat_1 = treecorr.Catalog(filename1, config_IA_noisefree, flip_g2=True,first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_IA_noisefree, flip_g2=True,first_row=2)
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2,metric = 'Arc')
        #gg.write('correlations/infused_cross_correlation_no_noise_tomo{}_{}'.format(j,i))

        gg.write('correlations/infused_cross_correlation_no_noise_tomo{}_{}_linear_bias'.format(j,i))

        #gg.write('correlations/infused_cross_correlation_no_noise_tomo_delta-NLA{}_{}'.format(j,i))
        #gg.write('correlations/infused_cross_correlation_no_noise_tomo_TATT{}_{}'.format(j,i))

        print("Done tomo", i,j)



Done tomo 2 1
Done tomo 3 1
Done tomo 3 2
Done tomo 4 1
Done tomo 4 2
Done tomo 4 3
Done tomo 5 1
Done tomo 5 2
Done tomo 5 3
Done tomo 5 4


In [12]:
# Run treecorr on the pure IA signal:

# Run the auto-correlation functions:
for i in range(1,6):
#for i in np.arange(5)+1:

    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
    filename = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)

    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_V3.dat'.format(i)
    #filename = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)



    cat = treecorr.Catalog(filename, config_pureIA, flip_g2=True,first_row=2)

    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    #gg.write('correlations/infused_cross_correlation_TATT_tomo{}_{}'.format(i,i))    
    gg.write('correlations/infused_cross_correlation_TATT_tomo{}_{}_linear_bias'.format(i,i))    

    #gg.write('correlations/infused_cross_correlation_TATT_tomo_delta-NLA{}_{}'.format(i,i))    
    #gg.write('correlations/infused_cross_correlation_TATT_tomo_delta-NLA{}_{}_V3'.format(i,i))    
    #gg.write('correlations/infused_cross_correlation_TATT_tomo_TATT{}_{}'.format(i,i))    

    print("Done tomo",i)

Done tomo 1
Done tomo 2
Done tomo 3
Done tomo 4
Done tomo 5


In [17]:
# And the cross-redshift bins
for i in np.arange(5)+1:
    for j in range(1, i):
        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(j)

        filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)
        filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(j)


        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(j)

        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(j)

        cat_1 = treecorr.Catalog(filename1, config_pureIA, flip_g2=True,first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_pureIA, flip_g2=True,first_row=2)
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2, metric = 'Arc')
        #gg.write('correlations/infused_cross_correlation_TATT_tomo{}_{}'.format(j,i))       
        gg.write('correlations/infused_cross_correlation_TATT_tomo{}_{}_linear_bias'.format(j,i))       

        #gg.write('correlations/infused_cross_correlation_TATT_tomo_delta-NLA{}_{}'.format(j,i))
        #gg.write('correlations/infused_cross_correlation_TATT_tomo_TATT{}_{}'.format(j,i))

        print("Done tomo",i,j)

Done tomo 2 1
Done tomo 3 1
Done tomo 3 2
Done tomo 4 1
Done tomo 4 2
Done tomo 4 3
Done tomo 5 1
Done tomo 5 2
Done tomo 5 3
Done tomo 5 4


In [3]:
# Get the GI term: 
#Run treecorr on the pure IA_TATT signal of the foreground, and the pure shear of the background:
# Get auto+cross bins
#for i in range(5,6):
#    for j in range(i+1, 6):
#for i in np.arange(5)+1:
for i in range(1,2):
    for j in range(1,i+1):

        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}.dat'.format(j)

        filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(i)
        filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_IA_{}_linear_bias.dat'.format(j)


        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}.dat'.format(j)
        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_V3.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_delta-NLA_{}_V3.dat'.format(j)


        #filename1 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(i)
        #filename2 = '/global/cscratch1/sd/jharno/fin_epsilon_TATT_{}.dat'.format(j)

        cat_1 = treecorr.Catalog(filename1, config_G, flip_g2=True,first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_pureIA, flip_g2=True,first_row=2)
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2, metric = 'Arc')
        #gg.write('correlations/infused_cross_correlation_GI_tomo{}_{}'.format(j,i))
        gg.write('correlations/infused_cross_correlation_GI_tomo{}_{}_linear_bias'.format(j,i))

        #gg.write('correlations/infused_cross_correlation_GI_tomo_delta-NLA{}_{}'.format(j,i))
        #gg.write('correlations/infused_cross_correlation_GI_tomo_delta-NLA{}_{}_V3'.format(j,i))
        #gg.write('correlations/infused_cross_correlation_GI_tomo_TATT{}_{}'.format(j,i))

        print("Done tomo",i,j)

/global/common/software/lsst/common/miniconda/dev-v1r1/envs/desc/lib/python3.9/site-packages/treecorr/ggcorrelation.py:510: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.varxip), np.sqrt(self.varxim),


Done tomo 1 1


In [4]:
1+1


2